In [1]:
# Importação de bibliotecas
import pandas as pd

from pendulum import now

In [2]:
# Objetos
fonte = 'sidra'
dataset_name = 'tab5457_sementes'
variavel = 'area_plantada'

raiz = '/home/eduardo/repositorios/IS_Agro/databases'

Área de colheita/plantada (ha)

### Bronze

In [ ]:
contador = 1

ultimo_ano = now().year - 1
for ano in range(2000, ultimo_ano):
    if contador == 1:
        df_5457 = pd.read_json(f'https://apisidra.ibge.gov.br/values/t/5457/n6/all/v/216/p/{ano}/c782/40102,40106,40112,40119,40122,40124,40127')
        contador += 1
    else:
        df_5457 = pd.concat(
            [df_5457,
            (pd.read_json(f'https://apisidra.ibge.gov.br/values/t/5457/n6/all/v/216/p/{ano}/c782/40102,40106,40112,40119,40122,40124,40127')
             .iloc[1:])],
            ignore_index=True            
        )
        contador += 1

df_5457.to_json(f'{raiz}/bronze_{fonte}_{dataset_name}_{variavel}.json')
print(f'Foram contabilizadas {contador} repetições.')

### Prata

In [5]:
df = (pd.read_json(f'{raiz}/bronze_{fonte}_{dataset_name}_{variavel}.json')
 .drop(0, axis = 0)
 .filter(['D1C', 'D3C', 'D4C', 'V'])
 .assign(**{
        'uf': lambda df: df['D1C'].str.slice(0,2).astype(int),
        'hectare': lambda df: pd.to_numeric(df['V'], errors='coerce'),
        'municipio': lambda df: df['D1C'].astype(int),
        'data': lambda df: pd.to_datetime(df['D3C'], format='%Y'), # Coluna "data" em datetime baseado na ISO 8601, convertido da coluna "D3C"
        'cultura': lambda df: df['D4C'].astype(int)
        })
 .dropna(subset='hectare')
 .convert_dtypes()
 .groupby(['uf', 'municipio', 'data', 'cultura'])
 .agg({'hectare':'sum'})
 )

df.to_parquet(f'{raiz}/prata_{fonte}_{dataset_name}_{variavel}.parquet')

### Ouro

Quantidade de sementes/mudas utilizadas por área para a plantação das principais culturas agrícolas do país.

((Área * Quantidade_Sementes_ha)/1000) * (Teor N,P,K/1000)

In [6]:
parametro = {
    40102: {'semente_ha': 160, 'n': 13, 'p': 2.9, 'k': 3}, # Arroz
    40106: {'semente_ha': 12500, 'n': 1, 'p': 0.3, 'k': 1.5}, # Cana-de-açúcar
    40112: {'semente_ha': 45, 'n': 50, 'p': 5.7, 'k': 12.5}, # Feijão
    40119: {'semente_ha': 750, 'n': 2.1, 'p': 0.2, 'k': 2}, # Mandioca
    40122: {'semente_ha': 23, 'n': 12, 'p': 2.7, 'k': 3.7}, # Milho
    40124: {'semente_ha': 70, 'n': 55, 'p': 5.2, 'k': 16.6}, # Soja
    40127: {'semente_ha': 120, 'n': 21, 'p': 4.1, 'k': 4.6}, # Trigo
    }

In [7]:
df = (pd.read_parquet(f'{raiz}/prata_{fonte}_{dataset_name}_{variavel}.parquet')
      .groupby(['uf', 'data', 'cultura'])
      .agg({'hectare':sum})
      .reset_index()
      .assign(**{
          'fonte': 'Sementes',
          'N': lambda df: df.apply(lambda lin: ((lin['hectare']*parametro[lin['cultura']]['semente_ha'])/1000) * (parametro[lin['cultura']]['n']/1000), axis = 1),
          'P': lambda df: df.apply(lambda lin: ((lin['hectare']*parametro[lin['cultura']]['semente_ha'])/1000) * (parametro[lin['cultura']]['p']/1000), axis = 1),
          'K': lambda df: df.apply(lambda lin: ((lin['hectare']*parametro[lin['cultura']]['semente_ha'])/1000) * (parametro[lin['cultura']]['k']/1000), axis = 1)
          })
      .drop('hectare', axis = 1)
      .melt(id_vars=['uf', 'data', 'cultura', 'fonte'],
            value_vars=['N', 'P', 'K'],
            var_name='elemento',
            value_name='valor')
      .convert_dtypes()
      .groupby(['uf', 'data', 'fonte', 'elemento'])
      .agg({'valor': sum})
      .reset_index(level='elemento')
      .sort_values(by=['elemento', 'uf'],
                   key=lambda x: x.map({'N': 0, 'P': 1, 'K': 2}))
)

/tmp/ipykernel_88291/1121279783.py:3: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  .agg({'hectare':sum})
/tmp/ipykernel_88291/1121279783.py:18: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  .agg({'valor': sum})


In [8]:
df = (
    df
    .rename_axis(index={
        'uf':'state',
        'data': 'date',
        'fonte': 'label'
    })
    .rename(columns={
        'elemento':'nutrient',
        'valor':'value'
    })
)

df.to_csv(f'{raiz}/ouro_npk_sementes.csv')